In [1]:
from msedge.selenium_tools import Edge, EdgeOptions
from selenium.webdriver.common.by import By
#from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
options = EdgeOptions()
options.use_chromium = True
options.add_argument("headless")
driver = Edge(executable_path=r'C:\dev\WebDrivers\edgedriver_win64_111\msedgedriver.exe',options=options)

# Listing search

## Trial

In [ ]:
driver.get('https://suumo.jp/ms/chuko/fukuoka/city/')

In [ ]:
search_box = driver.find_element(By.CLASS_NAME, 'searchbox')

In [ ]:
search_button = search_box.find_element(By.TAG_NAME,'a').click()

In [ ]:
#properties = driver.find_elements(By.XPATH, "//div[starts-with(@class,'property_unit ')]")
properties = driver.find_elements(By.XPATH, "//div[@class='property_unit-content']")

In [ ]:
prop = properties[0]
prop_info = prop.find_elements(By.CLASS_NAME, 'dottable-line')
price = prop_info[1].find_element(By.CLASS_NAME, 'dottable-value').text
print(price)
address = prop_info[2].find_element(By.TAG_NAME, 'dd').text
print(address)
closest = prop_info[2].find_elements(By.TAG_NAME, 'dd')[1].text.split('徒歩',1)[1][:-1]
print(closest)
area = prop_info[3].find_element(By.TAG_NAME, 'dd').text.split('m',1)[0]
print(area)
ap_type = prop_info[3].find_elements(By.TAG_NAME, 'dd')[1].text
print(ap_type)
try:
    balcony = float(prop_info[4].find_element(By.TAG_NAME, 'dd').text.split('m',1)[0])
except ValueError:
    balcony = 0
print(balcony)
date = prop_info[4].find_elements(By.TAG_NAME, 'dd')[1].text[:4]
print(date)

In [ ]:
prop = properties[0]
prop_link = prop.find_element(By.TAG_NAME, 'a')
prop_id = prop_link.get_attribute('href').split('/')[-2]
print(prop_id)

In [ ]:
index = address.find('県')
if index == -1 or index >2:
    index = address.find('府')
if index == -1 or index >2:
    index = address.find('道')
if index == -1 or index >2:
    index = address.find('都')
    tokyo=True
prefecture = address[:index]
address = address[index+1:]
index = address.find('市')
if index == -1:    
    index = address.find('区')
    special_ward = True
else:
    special_ward = False
city = address[:index]
address = address[index+1:]
if special_ward:
    ward = city
else:
    index = address.find('区')
    ward = address[:index]
    address = address[index+1:]
neighborhood = address
print(prefecture, city, ward, neighborhood)

## Scrapping all search query response pages

In [ ]:
driver = Edge(executable_path=r'C:\dev\WebDrivers\edgedriver_win64_111\msedgedriver.exe',options=options)
driver.get('https://suumo.jp/ms/chuko/fukuoka/city/')
search_box = driver.find_element(By.CLASS_NAME, 'searchbox')
search_button = search_box.find_element(By.TAG_NAME,'a').click()
        
col_names =  ['Link', 'Prefecture', 'City', 'Ward', 'Neighborhood', 'Type', 'Area m2', 'Balcony m2', 'Closest station (min)', 'Construction year', 'Price']
n=0

while True:
    try:
        print('Scraping page {}.\n'.format(n))
        properties = driver.find_elements(By.XPATH, "//div[@class='property_unit-content']")
        prop_link=[]
        price=[]
        area=[]
        ap_type=[]
        balcony=[]
        year=[]
        prefecture=[]
        city=[]
        ward=[]
        neighborhood=[]
        for prop in properties:
            link = prop.find_element(By.TAG_NAME, 'a')
            prop_link.append(link.get_attribute('href'))

            prop_info = prop.find_elements(By.CLASS_NAME, 'dottable-line')
            price.append(prop_info[1].find_element(By.CLASS_NAME, 'dottable-value').text)
            address = prop_info[2].find_element(By.TAG_NAME, 'dd').text
            area.append(prop_info[3].find_element(By.TAG_NAME, 'dd').text.split('m',1)[0])
            ap_type.append(prop_info[3].find_elements(By.TAG_NAME, 'dd')[1].text)
            try:
                balc = float(prop_info[4].find_element(By.TAG_NAME, 'dd').text.split('m',1)[0])
            except ValueError:
                balc = 0
            balcony.append(balc)
            year.append(prop_info[4].find_elements(By.TAG_NAME, 'dd')[1].text[:4])
            index = address.find('県')
            if index == -1 or index >3:
                index = address.find('府')
            if index == -1 or index >2:
                index = address.find('道')
            if index == -1 or index >2:
                index = address.find('都')
                tokyo=True
            prefecture.append(address[:index])
            address = address[index+1:]
            index = address.find('市')
            if index == -1:    
                index = address.find('区')
                special_ward = True
            else:
                special_ward = False
            city.append(address[:index])
            address = address[index+1:]
            if special_ward:
                ward.append(city[-1])
            else:
                index = address.find('区')
                ward.append(address[:index])
                address = address[index+1:]
            neighborhood.append(address)
        try:
            properties_df = properties_df.append(pd.DataFrame(list(zip(prop_link, prefecture, city, ward, neighborhood, ap_type, area, balcony, year, price)), columns = col_names, ignore_index=True))
        except NameError:
            properties_df = pd.DataFrame(list(zip(prop_link, prefecture, city, ward, neighborhood, ap_type, area, balcony, year, price)), columns = col_names)

        pagination = driver.find_elements(By.XPATH, "//p[@class='pagination-parts']")
        if pagination[-1].text == "次へ":
            pagination[-1].find_element(By.TAG_NAME,'a').click()
            n = n+1
        else:
            break
    except:
        driver = Edge(executable_path=r'C:\dev\WebDrivers\edgedriver_win64_111\msedgedriver.exe',options=options)
        driver.get('https://suumo.jp/ms/chuko/fukuoka/city/')
        search_box = driver.find_element(By.CLASS_NAME, 'searchbox')
        search_button = search_box.find_element(By.TAG_NAME,'a').click() 
        driver.get(driver.current_url+"&page"+str(n))
        print('Restored and got back to page {}.'.format(driver.find_element(By.CLASS_NAME, 'pagination-current').text))
            
properties_df.drop_duplicates()
properties_df.to_csv('bukken_data.csv')

In [ ]:
properties_df.size

In [ ]:
properties_df.shape[0]

In [ ]:
properties_df.head()

In [ ]:
pagination = driver.find_element(By.XPATH, "//ol[@class='pagination-parts']")
pages= pagination.find_elements(By.TAG_NAME, "li")
for page in pages:print(page.text)

In [ ]:
if pagination[-1].text == "次へ":
    pagination[-1].find_element(By.TAG_NAME,'a').click()

In [ ]:
current_page = driver.find_element(By.CLASS_NAME, 'pagination-current').text
print(current_page)
print(pagination[3].text)

In [ ]:
if 'properties_df' in locals():
    del properties_df

In [ ]:
properties_df.to_csv('bukken_data.csv')

In [ ]:
n = 13

driver.get('https://suumo.jp/ms/chuko/fukuoka/city/')
search_box = driver.find_element(By.CLASS_NAME, 'searchbox')
search_button = search_box.find_element(By.TAG_NAME,'a').click()
pagination = driver.find_element(By.XPATH, "//div[@class='pagination pagination_set-nav']")
pages= pagination.find_elements(By.TAG_NAME, "li")
current_page = int(driver.find_element(By.CLASS_NAME, 'pagination-current').text)
while  current_page!= n:
    print(current_page)
    if int(pages[-3].text) < n:
        pages[-3].find_element(By.TAG_NAME,'a').click()
    else:
        driver.find_elements(By.XPATH, "//p[@class='pagination-parts']")[-1].find_element(By.TAG_NAME,'a').click()
    pagination = driver.find_element(By.XPATH, "//div[@class='pagination pagination_set-nav']")
    pages= pagination.find_elements(By.TAG_NAME, "li")
    current_page = int(driver.find_element(By.CLASS_NAME, 'pagination-current').text)
print('I have found page {}'.format(current_page))

In [ ]:
len(pages)

In [ ]:
print(driver.find_element(By.CLASS_NAME, 'pagination-current').text)
driver.find_elements(By.XPATH, "//p[@class='pagination-parts']")[-1].find_element(By.TAG_NAME,'a').click()
print(driver.find_element(By.CLASS_NAME, 'pagination-current').text)

In [ ]:
pagination = driver.find_element(By.XPATH, "//div[@class='pagination pagination_set-nav']")

In [ ]:
pages= pagination.find_elements(By.TAG_NAME, "li")
for page in pages:  print(page.text)

In [ ]:
driver.current_url+"&page"+str(n)

# Scrapping individual listing

In [3]:
driver.get('https://suumo.jp/ms/chuko/fukuoka/sc_fukutsu/nc_70625415/')

In [4]:
data_table = driver.find_element(By.XPATH, "//table[@summary='表']").find_elements(By.TAG_NAME, 'td')

In [39]:
for i, e in enumerate(data_table): print(str(i) + ': ' + e.text)

0: クレスト福間駅前　【ホームスターク住まい情報館】
1: 2190万円
[ □支払シミュレーション ]
2: 4LDK
3: 1戸
4: 87戸
5: 85.84m2（25.96坪）（壁芯）
6: バルコニー面積：33.89m2
7: 13階/SRC14階建
8: 1991年11月
9: 福岡県福津市中央６
[ ■周辺環境 ]
10: ＪＲ鹿児島本線「福間」歩3分
[ 乗り換え案内 ]
ＪＲ鹿児島本線「千鳥」歩32分
[ 乗り換え案内 ]
ＪＲ鹿児島本線「東福間」歩37分
[ 乗り換え案内 ]
11: 【この会社の関連サイト】
クラシーク福岡（弊社ホームページ）
OUCA（弊社ホームページ）
12: 担当者
宅建
豊島　賢広
年齢：30代
業界経験：15年
お客様の満足を第一に、お客様の視点で発想し、より満足される方法を、考えて判断を行います。お客様に納得・満足していただけるように誠意と努力をもって対応致します！なんなりとお申し付けください！
13: 【この物件について】 駅チカ物件☆スーパー・コンビニ徒歩圏内！幼稚園、小・中学校へも近く、快適に暮らせる立地。お仕事帰りや、お買い物でお近くにいらっしゃる際にぜひお越し下さい！平日のご見学も可能です、お気軽にご相談下さい。
【得意エリア】 福岡都市圏全域
14: (株)ホームスターク本店
TEL：0800-814-0011 【通話料無料】
携帯電話・ＰＨＳからもご利用いただけます

「SUUMO(スーモ)を見た」と問い合わせください

つながらない方、不動産会社の方はこちら
15: 免許番号：福岡県知事（2）第017482号
取引態様：＜売主＞
営業時間：9:00～20:00 ／ 定休日：第一、第二火曜日・水曜日


In [146]:
def individual_listing(links):
    full_address = []
    floor = []
    max_floor = []
    building_type = []
    closest = []
    nb_station = []
    
    while True:
        try:
            for link in links[len(full_address):]:
                print('Listing number {}'.format(len(full_address)))
                driver.get(link)
                #print('got page')
                try:
                    data_table = driver.find_element(By.XPATH, "//table[@summary='表']").find_elements(By.TAG_NAME, 'td')
                except:
                    #print('Table not found page')
                    full_address.append(None)
                    floor.append(None)
                    max_floor.append(None)
                    building_type.append(None)
                    closest.append(None)
                    nb_station.append(None)
                    continue
                #print('got table')
                full_address.append(data_table[9].text.split('\n', 1)[0])
                #print('address')
                sep = data_table[7].text.find('/')
                floor.append(data_table[7].text[:sep-1])
                #print('floor')
                building_info = data_table[7].text[sep+1:]
                try:
                    max_floor.append(int(building_info[building_info.find('階建')-2:building_info.find('階建')]))
                    building_type.append(''.join([building_info[:building_info.find('階建')-2],building_info[building_info.find('階建')+2:]]))
                except:
                    max_floor.append(int(building_info[building_info.find('階建')-1:building_info.find('階建')]))
                    building_type.append(''.join([building_info[:building_info.find('階建')-1],building_info[building_info.find('階建')+2:]]))
                #print('building')
                time = 60
                stations = 0
                transportation = data_table[10].text.splitlines()
                for transport in transportation:
                    if transport[-1]!= '分':
                        continue
                    else:
                        stations +=1
                        try:
                            distance = int(transport.split('歩',1)[1][:-1])
                            time = min(time, distance)
                        except IndexError:
                            pass
                closest.append(time)
                nb_station.append(stations)
                #print('transport')
            
            return full_address, floor, max_floor, building_type, closest, nb_station
        
        except:
            print('Other exception')
            driver = Edge(executable_path=r'C:\dev\WebDrivers\edgedriver_win64_111\msedgedriver.exe',options=options)
            
            

In [163]:
listing_data = pd.read_csv('bukken_data.csv', index_col=0)
listing_data.shape

(9305, 10)

In [66]:
sample_listing = listing_data.head(5)

In [67]:
sample_listing

,Link,Prefecture,City,Ward,Neighborhood,Type,Area m2,Balcony m2,Construction year,Price
0,https://suumo.jp/ms/chuko/fukuoka/sc_kitakyush...,福岡,北九州,門司,清見１,4LDK,75.96,12.75,1983,250万円
1,https://suumo.jp/ms/chuko/fukuoka/sc_fukuokash...,福岡,福岡,南,高木２,3LDK,63.57,9.79,1976,580万円
2,https://suumo.jp/ms/chuko/fukuoka/sc_fukuokash...,福岡,福岡,博多,西月隈１,3LDK,60.61,8.10,1983,590万円
3,https://suumo.jp/ms/chuko/fukuoka/sc_kitakyush...,福岡,北九州,若松,童子丸１,2LDK,52.66,8.24,1986,598万円
4,https://suumo.jp/ms/chuko/fukuoka/sc_kitakyush...,福岡,北九州,若松,白山１,2LDK,55.18,5.41,1979,698万円


In [55]:
sample_listing['Link'].tolist()

['https://suumo.jp/ms/chuko/fukuoka/sc_kitakyushushimoji/nc_71981425/',
 'https://suumo.jp/ms/chuko/fukuoka/sc_fukuokashiminami/nc_71941193/',
 'https://suumo.jp/ms/chuko/fukuoka/sc_fukuokashihakata/nc_70870669/',
 'https://suumo.jp/ms/chuko/fukuoka/sc_kitakyushushiwakamatsu/nc_71726165/',
 'https://suumo.jp/ms/chuko/fukuoka/sc_kitakyushushiwakamatsu/nc_71079749/']

In [68]:
full_address=['a', 'b', 'c',None,'e']
floor= [1,2,3,None,5]
max_floor=[1,2,3,None,5]
building_type=['a', 'b', 'c',None,'e']
closest=[1,2,3,None,5]
nb_station=[1,2,3,None,5] 
sample_listing.insert(1,'Full address', full_address, True)
sample_listing.insert(sample_listing.shape[1]-1,'Floor', floor, True)
sample_listing.insert(sample_listing.shape[1]-1,'Highest floor', max_floor, True)
sample_listing.insert(sample_listing.shape[1]-1,'Building type', building_type, True)
sample_listing.insert(sample_listing.shape[1]-1,'Closest station (min)', closest, True)
sample_listing.insert(sample_listing.shape[1]-1,'Number of stations', nb_station, True)
sample_listing

,Link,Full address,Prefecture,City,Ward,Neighborhood,Type,Area m2,Balcony m2,Construction year,Floor,Highest floor,Building type,Closest station (min),Number of stations,Price
0,https://suumo.jp/ms/chuko/fukuoka/sc_kitakyush...,a,福岡,北九州,門司,清見１,4LDK,75.96,12.75,1983,1.0,1.0,a,1.0,1.0,250万円
1,https://suumo.jp/ms/chuko/fukuoka/sc_fukuokash...,b,福岡,福岡,南,高木２,3LDK,63.57,9.79,1976,2.0,2.0,b,2.0,2.0,580万円
2,https://suumo.jp/ms/chuko/fukuoka/sc_fukuokash...,c,福岡,福岡,博多,西月隈１,3LDK,60.61,8.10,1983,3.0,3.0,c,3.0,3.0,590万円
3,https://suumo.jp/ms/chuko/fukuoka/sc_kitakyush...,None,福岡,北九州,若松,童子丸１,2LDK,52.66,8.24,1986,NaN,NaN,None,NaN,NaN,598万円
4,https://suumo.jp/ms/chuko/fukuoka/sc_kitakyush...,e,福岡,北九州,若松,白山１,2LDK,55.18,5.41,1979,5.0,5.0,e,5.0,5.0,698万円


In [160]:
p_full_address, p_floor, p_max_floor, p_building_type, p_closest, p_nb_station = individual_listing(listing_data['Link'].tolist()[7500:])
print(len(p_full_address))
print(len(p_floor))
print(len(p_max_floor))
print(len(p_building_type))
print(len(p_closest))
print(len(p_nb_station))

Listing number 0
Other exception
Listing number 0
Listing number 1
Listing number 2
Listing number 3
Listing number 4
Listing number 5
Listing number 6
Listing number 7
Listing number 8
Listing number 9
Listing number 10
Listing number 11
Listing number 12
Listing number 13
Listing number 14
Listing number 15
Listing number 16
Listing number 17
Listing number 18
Listing number 19
Listing number 20
Listing number 21
Listing number 22
Listing number 23
Listing number 24
Listing number 25
Listing number 26
Listing number 27
Listing number 28
Listing number 29
Listing number 30
Listing number 31
Listing number 32
Listing number 33
Listing number 34
Listing number 35
Listing number 36
Listing number 37
Listing number 38
Listing number 39
Listing number 40
Listing number 41
Listing number 42
Listing number 43
Listing number 44
Listing number 45
Listing number 46
Listing number 47
Listing number 48
Listing number 49
Listing number 50
Listing number 51
Listing number 52
Listing number 53
Listi

Listing number 434
Listing number 435
Listing number 436
Listing number 437
Listing number 438
Listing number 439
Listing number 440
Listing number 441
Listing number 442
Listing number 443
Listing number 444
Listing number 445
Listing number 446
Listing number 447
Listing number 448
Listing number 449
Listing number 450
Listing number 451
Listing number 452
Listing number 453
Listing number 454
Listing number 455
Listing number 456
Listing number 457
Listing number 458
Listing number 459
Listing number 460
Listing number 461
Listing number 462
Listing number 463
Listing number 464
Listing number 465
Listing number 466
Listing number 467
Listing number 468
Listing number 469
Listing number 470
Listing number 471
Listing number 472
Listing number 473
Listing number 474
Listing number 475
Listing number 476
Listing number 477
Listing number 478
Listing number 479
Listing number 480
Listing number 481
Listing number 482
Listing number 483
Listing number 484
Listing number 485
Listing numb

Listing number 866
Listing number 867
Listing number 868
Listing number 869
Listing number 870
Listing number 871
Listing number 872
Listing number 873
Listing number 874
Listing number 875
Listing number 876
Listing number 877
Listing number 878
Listing number 879
Listing number 880
Listing number 881
Listing number 882
Listing number 883
Listing number 884
Listing number 885
Listing number 886
Listing number 887
Listing number 888
Listing number 889
Listing number 890
Listing number 891
Listing number 892
Listing number 893
Listing number 894
Listing number 895
Listing number 896
Listing number 897
Listing number 898
Listing number 899
Listing number 900
Listing number 901
Listing number 902
Listing number 903
Listing number 904
Listing number 905
Listing number 906
Listing number 907
Listing number 908
Listing number 909
Listing number 910
Listing number 911
Listing number 912
Listing number 913
Listing number 914
Listing number 915
Listing number 916
Listing number 917
Listing numb

Listing number 1281
Listing number 1282
Listing number 1283
Listing number 1284
Listing number 1285
Listing number 1286
Listing number 1287
Listing number 1288
Listing number 1289
Listing number 1290
Listing number 1291
Listing number 1292
Listing number 1293
Listing number 1294
Listing number 1295
Listing number 1296
Listing number 1297
Listing number 1298
Listing number 1299
Listing number 1300
Listing number 1301
Listing number 1302
Listing number 1303
Listing number 1304
Listing number 1305
Listing number 1306
Listing number 1307
Listing number 1308
Listing number 1309
Listing number 1310
Listing number 1311
Listing number 1312
Listing number 1313
Listing number 1314
Listing number 1315
Listing number 1316
Listing number 1317
Listing number 1318
Listing number 1319
Listing number 1320
Listing number 1321
Listing number 1322
Listing number 1323
Listing number 1324
Listing number 1325
Listing number 1326
Listing number 1327
Listing number 1328
Listing number 1329
Listing number 1330


Listing number 1691
Listing number 1692
Listing number 1693
Listing number 1694
Listing number 1695
Listing number 1696
Listing number 1697
Listing number 1698
Listing number 1699
Listing number 1700
Listing number 1701
Listing number 1702
Listing number 1703
Listing number 1704
Listing number 1705
Listing number 1706
Listing number 1707
Listing number 1708
Listing number 1709
Listing number 1710
Listing number 1711
Listing number 1712
Listing number 1713
Listing number 1714
Listing number 1715
Listing number 1716
Listing number 1717
Listing number 1718
Listing number 1719
Listing number 1720
Listing number 1721
Listing number 1722
Listing number 1723
Listing number 1724
Listing number 1725
Listing number 1726
Listing number 1727
Listing number 1728
Listing number 1729
Listing number 1730
Listing number 1731
Listing number 1732
Listing number 1733
Listing number 1734
Listing number 1735
Listing number 1736
Listing number 1737
Listing number 1738
Listing number 1739
Listing number 1740


In [147]:
full_address = []
floor = []
max_floor = []
building_type = []
closest = []
nb_station = []

In [167]:
full_address.append(p_full_address)
floor.append(p_floor)
max_floor.append(p_max_floor)
building_type.append(p_building_type)
closest.append(p_closest)
nb_station.append(p_nb_station)

In [175]:
full_address=sum(full_address,[])
floor=sum(floor,[])
max_floor=sum(max_floor,[])
building_type=sum(building_type,[])
closest=sum(closest,[])
nb_station=sum(nb_station,[])

In [185]:
listing_data.insert(1,'Full address', full_address, True)
listing_data.insert(listing_data.shape[1]-1,'Floor', floor, True)
listing_data.insert(listing_data.shape[1]-1,'Highest floor', max_floor, True)
listing_data.insert(listing_data.shape[1]-1,'Building type', building_type, True)
listing_data.insert(listing_data.shape[1]-1,'Closest station (min)', closest, True)
listing_data.insert(listing_data.shape[1]-1,'Number of stations', nb_station, True)
listing_data

,Link,Full address,Prefecture,City,Ward,Neighborhood,Type,Area m2,Balcony m2,Construction year,Floor,Highest floor,Building type,Closest station (min),Number of stations,Price
0,https://suumo.jp/ms/chuko/fukuoka/sc_kitakyush...,福岡県北九州市門司区清見１,福岡,北九州,門司,清見１,4LDK,75.96,12.75,1983,2,5.0,RC,4.0,2.0,250万円
1,https://suumo.jp/ms/chuko/fukuoka/sc_fukuokash...,福岡県福岡市南区高木２,福岡,福岡,南,高木２,3LDK,63.57,9.79,1976,5,5.0,RC,13.0,3.0,580万円
2,https://suumo.jp/ms/chuko/fukuoka/sc_fukuokash...,福岡県福岡市博多区西月隈１,福岡,福岡,博多,西月隈１,3LDK,60.61,8.10,1983,4,5.0,RC,24.0,3.0,590万円
3,https://suumo.jp/ms/chuko/fukuoka/sc_kitakyush...,福岡県北九州市若松区童子丸１,福岡,北九州,若松,童子丸１,2LDK,52.66,8.24,1986,2,7.0,RC,1.0,3.0,598万円
4,https://suumo.jp/ms/chuko/fukuoka/sc_kitakyush...,福岡県北九州市若松区白山１,福岡,北九州,若松,白山１,2LDK,55.18,5.41,1979,3,5.0,RC,3.0,2.0,698万円
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9300,https://suumo.jp/ms/chuko/fukuoka/sc_fukuokash...,None,福岡,福岡,中央,御所ヶ谷,4LDK,96.41,15.23,2005,None,NaN,None,NaN,NaN,4630万円
9301,https://suumo.jp/ms/chuko/fukuoka/sc_fukuokash...,福岡県福岡市東区香椎照葉３,福岡,福岡,東,香椎照葉３,3LDK,106.61,13.00,2016,36,45.0,RC,1.0,1.0,6800万円
9302,https://suumo.jp/ms/chuko/fukuoka/sc_fukuokash...,福岡県福岡市東区香椎照葉３,福岡,福岡,東,香椎照葉３,3LDK,106.61,13.00,2016,36,45.0,RC,4.0,2.0,6800万円
9303,https://suumo.jp/ms/chuko/fukuoka/sc_fukuokash...,福岡県福岡市早良区百道浜４,福岡,福岡,早良,百道浜４,5LDK,150.70,74.07,1992,1,8.0,RC,11.0,1.0,7680万円


## Get location data

In [2]:
import requests
import os

In [3]:
response = requests.get('https://dev.virtualearth.net/REST/v1/Locations?q=福岡県北九州市門司区清見１&culture=ja&key=' + os.environ['BING_MAP_API'])

KeyError: 'BING_MAP_API'

In [5]:
os.environ['BING_MAP_API']

KeyError: 'BING_MAP_API'